In [1]:
import pickle
import numpy as np
import pandas as pd
import math
import warnings
warnings.filterwarnings('ignore')

In [2]:
#1获取第一个用户打卡的combined_second_cat_id信息
#1.1获取所有用户combined_second_cat_id
with open('/home/jovyan/datasets/tsmc_nyc_6_distinguish_cat_poi/reindex_second_cat_id.pkl','rb') as f:
    users_combined_second_cat_id=pickle.load(f)
#1.2取第100个用户
user_combined_second_cat_id=users_combined_second_cat_id[0]
revers_user_combined_second_cat_id=user_combined_second_cat_id[::-1]
print(f'len of 100th user checkins：{len(user_combined_second_cat_id)}')
print(f'1st user checkin second id：{user_combined_second_cat_id}')
print(f'revers:{revers_user_combined_second_cat_id}')

len of 100th user checkins：59
1st user checkin second id：[0, 1, 2, 3, 3, 3, 4, 0, 4, 5, 4, 3, 3, 6, 4, 3, 2, 3, 7, 2, 4, 3, 0, 3, 0, 8, 6, 4, 1, 3, 2, 3, 3, 4, 1, 9, 4, 3, 3, 3, 3, 3, 3, 6, 3, 4, 10, 0, 4, 3, 10, 4, 4, 11, 3, 3, 11, 1, 11]
revers:[11, 1, 11, 3, 3, 11, 4, 4, 10, 3, 4, 0, 10, 4, 3, 6, 3, 3, 3, 3, 3, 3, 4, 9, 1, 4, 3, 3, 2, 3, 1, 4, 6, 8, 0, 3, 0, 3, 4, 2, 7, 3, 2, 3, 4, 6, 3, 3, 4, 5, 4, 0, 4, 3, 3, 3, 2, 1, 0]


In [3]:
'''以最后一个为测试，用之前的数据计算'''
#2.需要知道所有的种类
#2.1读所有的种类字典
with open('/home/jovyan/datasets/tsmc_nyc_6_distinguish_cat_poi/distin_dic_second_cat_id.pkl','rb') as f:
    distin_dic_second_cat_id=pickle.load(f)#combined_second_id:reindex
#2.2打卡中所出现second_cat_id个数
len_second_reindex=len(distin_dic_second_cat_id)#53
combined_second_reindex_id=list(distin_dic_second_cat_id.values())

In [4]:
# 3整个打卡序列的概率、53个打卡概率
# g_uc=[0]*len_second_reindex
# len_user_checkin=len(user_combined_second_cat_id)
# for checkin_second_cat in user_combined_second_cat_id:#循环用户打卡
#     if checkin_second_cat in combined_second_reindex_id:#如果这个打卡种类id在所有二级目录中
#         g_uc[combined_second_reindex_id.index(checkin_second_cat)]+=1#则在相应位置累加1
# g_uc=np.array(g_uc)/len_user_checkin#array
# print(g_uc.sum())

In [5]:
#3.2将概率封装成函数
def Compute_probability(checkins,second_cat_id):
    '''
    len_p:离散概率的长度,int
    checkins：用户的打卡列表，这里指的是所有打卡列表/月周期的打卡列表,list
    second_cat_id:所有的second_cat_id,用于找离散概率的相应位置,list
    return:prob,np.array
    '''
    prob=[0]*len(second_cat_id)
    len_checkin=len(checkins)#用于做概率相除的分母
    for c in checkins:
        prob[second_cat_id.index(c)]+=1
    fre_uc=prob#该用户在所有种类上的频度
    prob=np.array(prob)/len_checkin
    return prob,fre_uc
#测试结果
len_user_checkin=len(user_combined_second_cat_id)
g_uc,_=Compute_probability(user_combined_second_cat_id,combined_second_reindex_id)
print(g_uc)#对数组使用数组内置方法array.sum()，不要使用sum(array)否则有误差

[0.08474576 0.06779661 0.06779661 0.37288136 0.20338983 0.01694915
 0.05084746 0.01694915 0.01694915 0.01694915 0.03389831 0.05084746
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.        ]


In [6]:
#3.4二维列表计算每个子列表的频率
def Compute_period_prob(len_period,checkins_2d,second_cat_id):
    '''
    len_period:时期个数，int
    checkins_2d:二维打卡数据。每个时期对应的打卡种类，
    （长度为时期个数，时期内打卡长度），若某时期无打卡，为空列表。2d list
    second_cat_id:所有second_cat_id,用于找离散概率的相应位置，list
    return:pro_2d:如果某个时期没有打卡，则这个打卡概率保持空列表。(时期个数，所有二级列表的长度53)2d list
    '''
    pro_2d=[[0]]*len_period
    #对每个时期计算
    for index,j in enumerate(checkins_2d):
        if j:#若j不为空列表，则计算该列表内的概率
            j_pro,_=Compute_probability(j,second_cat_id)
            #将这个时期的概率，更新到2d_pro
            pro_2d[index]=j_pro
        if not j:
            pro_2d[index]=j
    return pro_2d

In [18]:
#4将用户的打卡分到每个月，得到时期的打卡概率
#4.1读取用户打卡的每月信息
with open('/home/jovyan/datasets/tsmc_nyc_3_groupby_user_chronological/month.pkl','rb') as f:
    months=pickle.load(f)
'''
foursquare的时间跨度为2012.4-2013.2，没有月数字重复
共11个月数字
'''
#4.2根据该用户所有打卡，将其分到11个时期中
user_month=months[0]
reversed_user_month=user_month[::-1]
span_month=[4,5,6,7,8,9,10,11,12,1,2]
period_second=[[]]*len(span_month)
for i in span_month:#遍历数据集中的时期
    if i in user_month:#如果用户在i月打卡
        #那么将该月打卡的second_id放到对应月列表的位置
        start=user_month.index(i)#得到切片起始
        end=len_user_checkin-reversed_user_month.index(i)#省去-1，因为下面切片，切片是右开区间
        period_second[span_month.index(i)]=user_combined_second_cat_id[start:end]
#4.3计算每个时期的打卡频率
# print(period_second)
len_period=len(span_month)
x_ujc=Compute_period_prob(len_period,period_second,combined_second_reindex_id)
print(x_ujc)

[array([0.16, 0.04, 0.12, 0.36, 0.2 , 0.04, 0.04, 0.04, 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  ]), array([0.        , 0.11111111, 0.11111111, 0.33333333, 0.22222222,
       0.        , 0.11111111, 0.        , 0.11111111, 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        

In [8]:
#5算法主体
#5.1
def Get_Cde_Guec(d,guc,combined_second_reindex_id):
    '''
    guc,combined_second_reindex_id:list,le=53
    return C_de:用户根据d划分的关键种类，G_uec:关键种类在用户所有打卡中的概率
    '''
    G_uec=[x for x in guc if x>=d]#先获得关键种类上的概率
    dc_mask=[1 if x>=d else 0 for x in guc]#对关键种类的位置取1，否则取0
    C_de=[val for val,m in zip(combined_second_reindex_id,dc_mask) if m]
    return G_uec,C_de,dc_mask
#5.1.1验证
G_uec,C_de,dc_mask=Get_Cde_Guec(0.00,g_uc,combined_second_reindex_id)
print(G_uec,'\n',C_de,'\n',len(G_uec))

[0.0847457627118644, 0.06779661016949153, 0.06779661016949153, 0.3728813559322034, 0.2033898305084746, 0.01694915254237288, 0.05084745762711865, 0.01694915254237288, 0.01694915254237288, 0.01694915254237288, 0.03389830508474576, 0.05084745762711865, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0] 
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52] 
 53


In [9]:
#5.2
def Get_xuec(d,x_ujc,dc_mask):
    '''
    获得在时期上关键种类的概率
    x_ujc:用户在j时期上关键种类的概率2d list,(j,53)
    dc_mask:整体上关键种类的位置mask
    return:x_ujec,在时期上关键种类的概率
    '''
    x_uec=[[val for val,m in zip(sub_ujc,dc_mask) if m] for sub_ujc in x_ujc]
    return x_uec
#5.2.1验证
G_uec,C_de,dc_mask=Get_Cde_Guec(0,g_uc,combined_second_reindex_id)
x_ujec=Get_xuec(0,x_ujc,dc_mask)
print(x_ujec[0])

[0.16, 0.04, 0.12, 0.36, 0.2, 0.04, 0.04, 0.04, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [10]:
#5.3
def KL(g_uec,x_ujec):
    '''
    x_ujec:1d list,j时期上关键种类的概率
    '''
    KLresult=0
    len_guec=len(g_uec)
    for gi,xi in zip(g_uec,x_ujec):
        if  xi != 0:
            m_ui=xi*(math.log(xi/gi))
        else:
            m_ui=math.inf
        KLresult+=m_ui
    return KLresult

In [22]:
#5.-1算法主题
def Compute_du(user_cat_id,g_uc,x_ujc,combined_second_reindex_id,span_month):
    '''
    user_cat_id:用户的打卡种类列表 1d_list
    g_uc:用户在所有种类的概率 1d list,len:53,所有种类个数
    x_ujc:用户在j时期上关键种类的概率2d list,(j,53)
    combined_second_reindex_id:所有的二级目录id，顺序和用户的打卡概率相同
    span_month:时期的跨度列表
    '''
    KLsum=0
    min=math.inf
    J=len(span_month)#时期跨度的长度
    for i in range(100):
        d=i/100
        G_uec,C_de,dc_mask=Get_Cde_Guec(d,g_uc,combined_second_reindex_id)#5.1
        
        sum_g=sum(G_uec)
        norm_G_uec=[val/sum_g for val in G_uec]#对G_uec归一化
        x_uec=Get_xuec(d,x_ujc,dc_mask)#5.2得到用户在时期关键种类上的概率
        for j in range(J):
            x_ujec=x_uec[j]#用户在某时期的概率
            
            sum_j=sum(x_ujec)
            norm_x_ujec=[val/sum_j for val in x_ujec]#对j时期关键种类概率归一化
            KLsum+=KL(norm_G_uec,norm_x_ujec)#5.4
        if KLsum<=min:
            min=KLsum
            du=d
    return du
#测试5.-1
du=Compute_du(user_combined_second_cat_id,g_uc,x_ujc,combined_second_reindex_id,span_month)
print(du)

0.06


In [12]:
#6获取关键种类
'''
对第100个用户取关键种类
'''
def Get_essential_second(du,g_uc,combined_second_reindex_id):
    '''
    取关键种类和相应mask
    g_uc:用户在种类上的打卡概率 1d_list，顺序和combined_second_reindex_id相同
    '''
    user_essential_second_pro=[val for val in g_uc if val>=du]
    #计算对应的种类
    user_essential_mask=[1 if val>=du else 0 for val in g_uc]
    user_essential_second=[val for val,m in zip(combined_second_reindex_id,user_essential_mask) if m]
    user_non_essential_second=[val for val,m in zip(combined_second_reindex_id,user_essential_mask) if not m]
    return user_essential_second,user_non_essential_second,user_essential_mask
user_essential_second,user_non_essential_second,user_essential_mask=Get_essential_second(du,g_uc,combined_second_reindex_id)#53个种类，缩减到5个种类,列表顺序是所有二级种类列表的顺序，不是用户打卡的顺序，用户打卡有57个
# print(len(user_essential_second),len(user_non_essential_second))

In [38]:
#7计算两类权重
def Compute_all_second_importance(user_essential_second,user_non_essential_second,combined_second_reindex_id,g_uc,user_combined_second_cat_id):
    second_importance=[0]*len(combined_second_reindex_id)
    for e in user_essential_second:
        index=combined_second_reindex_id.index(e)
        second_importance[index]=g_uc[index]
    for ne in user_non_essential_second:
        index=combined_second_reindex_id.index(ne)
        _,fre_uc=Compute_probability(user_combined_second_cat_id,combined_second_reindex_id)
        g_uci=g_uc[index]
        ln=math.log(len(user_combined_second_cat_id)/(1+fre_uc[index]))
        numerator=g_uci*ln
        denominator=0
        for i in range(len(user_non_essential_second)):
            denominator_inside=g_uc[i]*(math.log(len(user_combined_second_cat_id)/(1+fre_uc[i])))
            denominator+=denominator_inside
        second_importance[index]=numerator/denominator
    return second_importance
# second_importance=Compute_all_second_importance(user_essential_second,user_non_essential_second,combined_second_reindex_id,g_uc,user_combined_second_cat_id)
# print(len(second_importance))
# print(second_importance)
# print(user_essential_mask)

[0.0847457627118644, 0.06779661016949153, 0.06779661016949153, 0.3728813559322034, 0.2033898305084746, 0.03202108203289859, 0.07638881052609106, 0.03202108203289859, 0.03202108203289859, 0.03202108203289859, 0.05636962604391176, 0.07638881052609106, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [14]:
len(set([21, 9, 3, 3, 10, 12, 22, 3, 3, 3, 3, 2, 3, 13, 2, 19, 8, 13, 3, 27, 9, 3, 22, 3, 3, 10, 13, 3, 22, 3, 6, 6, 3, 6, 6, 13, 6, 3, 9, 3, 25, 3, 13, 25, 3, 19, 3, 25, 25, 3, 3, 0, 25, 9, 3, 3, 25]))

14

In [35]:
#8对一个用户所有打卡都做种类权重生成
#8.1算法主题
def User_importance_second(user_checkins,combined_second_id,span_month,user_months):
    '''
    user_checkins:一个用户的所有打卡，1d list
    combined_second_id:数据集中出现的second_id,1d_list
    span_month:数据集出现的月份时间跨度
    user_month:用户打卡的月份
    return:second_importance该用户所有的关键种类和对应权重，2d list，(时间步，二级种类个数)
    '''
    len_timestep=len(user_checkins)
    len_period=len(span_month)
    second_importance=[]
    #8.1循环该用户的时间步
    for i in range(1,len_timestep+1):#从0开始切得空列表，并且切不到最后一个序列，左闭右开列表
        present_checkin=user_checkins[:i]#对用户切片，得到用户[:i]种类的切片
        present_month=user_months[:i]
        len_present=i
        g_uc_present,_=Compute_probability(present_checkin,combined_second_id)#计算整体种类的概率
        #计算时期的概率
        period_second=Split_period(present_month,span_month,len_present,present_checkin)#将当前打卡划分到月份中
        x_uc=Compute_period_prob(len_period,period_second,combined_second_id)#计算月份中的打卡概率
        #计算此时打卡下的du
        du_present=Compute_du(present_checkin,g_uc,x_uc,combined_second_id,span_month)
        #获取关键种类
        user_essential_second_present,user_non_essential_present,user_essential_mask=Get_essential_second(du_present,g_uc_present,combined_second_id)
        #计算两类权重
        second_importance_present=Compute_all_second_importance(user_essential_second_present,user_non_essential_present,combined_second_id,g_uc_present,user_combined_second_cat_id)
        
        second_importance.append(second_importance_present)#更新时间步结果
    return second_importance
#8.2定义将此时打卡之前的数据分到月份里
def Split_period(present_month,span_month,len_present,present_checkin):
    '''
    return:period_second 2d list,用户在月份中打卡的种类id（月份，在该月份中的打卡），若在该月没有打卡，则该位置取[]
    '''
    reversed_present_month=present_month[::-1]
    period_second=[[]]*len(span_month)
    for i in span_month:
        if i in present_month:
            start=present_month.index(i)
            end=len_present-reversed_present_month.index(i)
            period_second[span_month.index(i)]=present_checkin[start:end]
    return period_second

# period_second=Split_period(months[0],span_month,len(months[0]),users_combined_second_cat_id[0])
# print(period_second)
# second_importance=User_importance_second(users_combined_second_cat_id[0],combined_second_reindex_id,span_month,months[0])
# print(len(second_importance))
# print(second_importance)

[[0, 1, 2, 3, 3, 3, 4, 0, 4, 5, 4, 3, 3, 6, 4, 3, 2, 3, 7, 2, 4, 3, 0, 3, 0], [8, 6, 4, 1, 3, 2, 3, 3, 4], [1, 9, 4, 3, 3, 3, 3, 3, 3, 6], [3, 4, 10, 0, 4, 3, 10, 4, 4, 11, 3], [3, 11], [], [], [], [1, 11], [], []]


In [37]:
# 9对所有用户计算每个时间步的权重-------------------------------------------------------ROM，using VSCode
# 9.1读取所有用户的打卡数据second，month(users_combined_second_cat_id,user_month)
def Compute_users_timesteps_importance(users_combined_second_cat_id,users_month,combined_second_reindex_id,span_month):
    '''
    users_combined_second_cat_id:2d list,(users,checkin_cat)
    user_month:2d list,(users,checkin_month)
    combined_second_reindex_id:1d list,数据集中所有的种类
    return: 3d list,(users,timestep,second)
    '''
    users_timesteps_importance=[]
    for u_s,u_m in zip(users_combined_second_cat_id,users_month):#循环每个用户
        u_timestep_importance=User_importance_second(u_s,combined_second_reindex_id,span_month,u_m)
        users_timesteps_importance.append(u_timestep_importance)
    return users_timesteps_importance

users_timesteps_importance=Compute_users_timesteps_importance(users_combined_second_cat_id,months,combined_second_reindex_id,span_month)
print(users_timesteps_importance[0])

KeyboardInterrupt: 

In [ ]:
# 10 保存所有用户在所有timestep上对所有种类的重要程度
dir_users_timesteps_second_importance='/home/jovyan/datasets/tsmc_nyc_10_users_timesteps_second_importance/users_timesteps_second_importance.pkl'
with open(dir_users_timesteps_second_importance,'wb') as f:
    pickle.dump(users_timesteps_importance,f)

In [30]:
a=[0,1,2,3]
a[:1]

[0]

In [32]:
for i in range(1,4):
    print(i)

1
2
3
